<a href="https://colab.research.google.com/github/felottermann/Coursera_Capstone/blob/main/whereTo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How Porto Alegre is clustered?

##### This project was done using the Foursquare API and it's a part of the IBM Coursera Data Science Specialization

### Importar as bibliotecas

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [ ]:
pip install geopandas

     |████████████████████████████████| 994 kB 28.3 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 
     |████████████████████████████████| 6.6 MB 62.7 MB/s 


### Importar o arquivo com os bairros

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving shapesbairros2016.zip to shapesbairros2016.zip


In [ ]:
import geopandas
import folium

In [ ]:
df = geopandas.read_file("shapesbairros2016.zip")
df.head()

,OBJECTID,CODIGO,EDITOR,DATA_EDICA,NOME,geometry
0,0,0,None,1899-12-30,LAMI,"POLYGON ((291163.025 1656544.013, 291173.664 1..."
1,0,0,None,1899-12-30,BOA VISTA DO SUL,"MULTIPOLYGON (((284220.343 1650812.330, 284221..."
2,0,0,None,1899-12-30,EXTREMA,"POLYGON ((294981.188 1654438.415, 294972.136 1..."
3,0,0,None,1899-12-30,BELÉM NOVO,"POLYGON ((284657.457 1658914.972, 284650.143 1..."
4,0,0,None,1899-12-30,SÃO CAETANO,"POLYGON ((294834.673 1658905.795, 294754.377 1..."


In [ ]:
df = df.to_crs(epsg=4326)
df['lat'] = df.centroid.y
df['lng'] = df.centroid.x
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


,OBJECTID,CODIGO,EDITOR,DATA_EDICA,NOME,geometry,lat,lng
0,0,0,None,1899-12-30,LAMI,"POLYGON ((-51.09178 -30.21069, -51.09167 -30.2...",-30.233499,-51.087802
1,0,0,None,1899-12-30,BOA VISTA DO SUL,"MULTIPOLYGON (((-51.16398 -30.26232, -51.16397...",-30.230662,-51.137142
2,0,0,None,1899-12-30,EXTREMA,"POLYGON ((-51.05214 -30.22970, -51.05223 -30.2...",-30.208037,-51.057805
3,0,0,None,1899-12-30,BELÉM NOVO,"POLYGON ((-51.15932 -30.18924, -51.15939 -30.1...",-30.208491,-51.175393
4,0,0,None,1899-12-30,SÃO CAETANO,"POLYGON ((-51.05364 -30.18940, -51.05447 -30.1...",-30.192859,-51.085067


### Um mapa dos bairros de Porto Alegre

In [ ]:
# m = folium.Map(location=[-30.03, -51.22], zoom_start=11, tiles='CartoDB positron')

# create map of Porto Alegre using latitude and longitude values
poa = folium.Map(location=[-30.0534301, -51.2299346], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['lat'], df['lng'], df['NOME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(poa)  
    
poa

### Foursquare Credentials and Version

In [ ]:
CLIENT_ID = '2XQIRUJ3AMSGTE2SGUZR0BP25JSK1RJLAW11WTSQW2DHHFTO' # your Foursquare ID
CLIENT_SECRET = 'MTG02QHYVDCXYTS3IA5PVVQPXY31UPZ3WGEYNFUZ5JIFWGNO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2XQIRUJ3AMSGTE2SGUZR0BP25JSK1RJLAW11WTSQW2DHHFTO
CLIENT_SECRET:MTG02QHYVDCXYTS3IA5PVVQPXY31UPZ3WGEYNFUZ5JIFWGNO


### Explorar o Data Frame

In [ ]:
df.loc[0, 'NOME']

'LAMI'

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2XQIRUJ3AMSGTE2SGUZR0BP25JSK1RJLAW11WTSQW2DHHFTO&client_secret=MTG02QHYVDCXYTS3IA5PVVQPXY31UPZ3WGEYNFUZ5JIFWGNO&v=20180605&ll=-30.063047218229258,-51.17698169595227&radius=500&limit=100'

In [ ]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60ff139a7e1193512bfaa2a8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4f8c62cfe4b09efba447edc2-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/cafeteria_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1a1941735',
         'name': 'College Cafeteria',
         'pluralName': 'College Cafeterias',
         'primary': True,
         'shortName': 'Cafeteria'}],
       'id': '4f8c62cfe4b09efba447edc2',
       'location': {'address': 'PUCRS',
        'cc': 'BR',
        'city': 'Porto Alegre',
        'country': 'Brasil',
        'crossStreet': 'Prédio 12',
        'distance': 331,
        'formattedAddress': ['PUCRS (Prédio 12)',
         'Porto Alegre, RS',
         '91530-000',
         'Brasil'],
        'l

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Bar do 12,College Cafeteria,-30.060293,-51.175667
1,Panorama Gastronômico,Brazilian Restaurant,-30.059182,-51.176033
2,Madero Container,Burger Joint,-30.059251,-51.179241
3,Metta Saúde Academia,Gym / Fitness Center,-30.063574,-51.178839
4,Body UP Academia,Gymnastics Gym,-30.062007,-51.173367


In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


### **2. Explorar os bairros em Porto Alegre**

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
poa_venues = getNearbyVenues(names=df['NOME'],latitudes=df['lat'],longitudes=df['lng'])

LAMI
BOA VISTA DO SUL
EXTREMA
BELÉM NOVO
SÃO CAETANO
PONTA GROSSA
CHAPÉU DO SOL
LAGEADO
SERRARIA
GUARUJÁ
ABERTA DOS MORROS
ESPÍRITO SANTO
HÍPICA
PITINGA
CAMPO NOVO
IPANEMA
JARDIM ISABEL
PEDRA REDONDA
RESTINGA
VILA CONCEIÇÃO
SÉTIMO CÉU
TRISTEZA
CAMAQUÃ
VILA NOVA
BELÉM VELHO
VILA  ASSUNÇÃO
CAVALHADA
CASCATA
NONOAI
CRISTAL
TERESÓPOLIS
GLÓRIA
ARQUIPÉLAGO
CORONEL APARÍCIO BORGES
VILA JOÃO PESSOA
VILA SÃO JOSÉ
SANTA TEREZA
MEDIANEIRA
AGRONOMIA
SANTO ANTÔNIO
JARDIM BOTÂNICO
JARDIM DO SALSO
MENINO DEUS
JARDIM CARVALHO
AZENHA
BOM JESUS
SANTA CECÍLIA
SANTANA
PRAIA DE BELAS
CIDADE BAIXA
FARROUPILHA
BOM FIM
MORRO SANTANA
PETRÓPOLIS
RIO BRANCO
CHÁCARA DAS PEDRAS
TRÊS FIGUEIRAS
BELA VISTA
JARDIM SABARÁ
VILA JARDIM
INDEPENDÊNCIA
MONTSERRAT
CENTRO HISTÓRICO
JARDIM EUROPA
MÁRIO QUINTANA
BOA VISTA
MOINHOS DE VENTO
AUXILIADORA
JARDIM LEOPOLDINA
PASSO DAS PEDRAS
HIGIENÓPOLIS
FLORESTA
VILA IPIRANGA
JARDIM ITU
PASSO DA AREIA
CRISTO REDENTOR
SANTA MARIA GORETTI
SÃO GERALDO
PARQUE SANTA FÉ
COSTA E SILVA
RUBEM

In [ ]:
print(poa_venues.shape)
poa_venues.head()

(1857, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BELÉM NOVO,-30.208491,-51.175393,Pub do Xavantes,-30.207530,-51.173161,Dive Bar
1,BELÉM NOVO,-30.208491,-51.175393,O Poderoso,-30.210908,-51.174835,Restaurant
2,BELÉM NOVO,-30.208491,-51.175393,Planet Dogs,-30.211090,-51.173707,Pet Store
3,BELÉM NOVO,-30.208491,-51.175393,Terminal 4:20,-30.211609,-51.178883,Plaza
4,PONTA GROSSA,-30.181949,-51.205803,Centro Social e de Treinamento Banrisul,-30.180449,-51.208752,Event Space


In [ ]:
poa_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ABERTA DOS MORROS,10,10,10,10,10,10
AGRONOMIA,4,4,4,4,4,4
ANCHIETA,10,10,10,10,10,10
ARQUIPÉLAGO,1,1,1,1,1,1
AUXILIADORA,88,88,88,88,88,88


How many unique categories?

In [ ]:
print('There are {} uniques categories.'.format(len(poa_venues['Venue Category'].unique())))

There are 250 uniques categories.


### Analyze Each Neighborhood

In [ ]:
# one hot encoding
poa_onehot = pd.get_dummies(poa_venues[['Venue Category']], prefix="", prefix_sep="")


In [ ]:

# add neighborhood column back to dataframe
poa_onehot['Neighborhood'] = poa_venues['Neighborhood'] 



In [ ]:
# move neighborhood column to the first column
fixed_columns = [poa_onehot.columns[-1]] + list(poa_onehot.columns[:-1])
poa_onehot = poa_onehot[fixed_columns]


In [ ]:
poa_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Baiano Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Campground,Candy Store,Champagne Bar,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empada House,Entertainment Service,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Latin American Restaurant,Lighting Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Event Space,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern Brazilian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Fair,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tapas Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BELÉM NOVO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0

In [ ]:
# one hot encoding
poa_onehot = pd.get_dummies(poa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
poa_onehot['Neighborhood'] = poa_venues['Neighborhood'] 

# move neighborhood column to the first column
# fixed_columns = [poa_onehot.columns[-1]] + list(poa_onehot.columns[:-1])
# poa_onehot = poa_onehot[fixed_columns]

first_column = poa_onehot['Neighborhood']
poa_onehot.insert(0, "Bairro", first_column)

poa_onehot.head()

,Bairro,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Baiano Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Campground,Candy Store,Champagne Bar,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empada House,Entertainment Service,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Latin American Restaurant,Lighting Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Event Space,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern Brazilian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Fair,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tapas Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,BELÉM NOVO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,BELÉM NOVO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
poa_grouped = poa_onehot.groupby('Bairro').mean().reset_index()
poa_grouped

,Bairro,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Baiano Restaurant,Bakery,Bank,Bar,Beach,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Campground,Candy Store,Champagne Bar,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empada House,Entertainment Service,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Latin American Restaurant,Lighting Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Event Space,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern Brazilian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Fair,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tapas Restaurant,Tapiocaria,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ABERTA DOS MORROS,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [ ]:
poa_grouped.shape

(85, 250)

In [ ]:
num_top_venues = 5

for hood in poa_grouped['Bairro']:
    print("----"+hood+"----")
    temp = poa_grouped[poa_grouped['Bairro'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABERTA DOS MORROS----
                  venue  freq
0                Bakery   0.2
1          Soccer Field   0.1
2           Bus Station   0.1
3              Bus Stop   0.1
4  Brazilian Restaurant   0.1


----AGRONOMIA----
               venue  freq
0   Stationery Store  0.25
1        Bus Station  0.25
2         Campground  0.25
3  College Cafeteria  0.25
4        Planetarium  0.00


----ANCHIETA----
                  venue  freq
0        Shipping Store   0.2
1  Fast Food Restaurant   0.1
2                 Motel   0.1
3            Restaurant   0.1
4   Rental Car Location   0.1


----ARQUIPÉLAGO----
                           venue  freq
0                 Gymnastics Gym   1.0
1              Accessories Store   0.0
2                    Pizza Place   0.0
3                Paintball Field   0.0
4  Paper / Office Supplies Store   0.0


----AUXILIADORA----
                  venue  freq
0  Gym / Fitness Center  0.07
1                  Café  0.06
2                   Bar  0.05
3           Piz

Let´s put that into a pandas dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Bairro']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Bairro'] = poa_grouped['Bairro']

for ind in np.arange(poa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(poa_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Bairro,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,ABERTA DOS MORROS,Bakery,Brazilian Restaurant,Soccer Field
1,AGRONOMIA,Stationery Store,Bus Station,College Cafeteria
2,ANCHIETA,Shipping Store,Rental Car Location,Nightclub
3,ARQUIPÉLAGO,Gymnastics Gym,Yoga Studio,Food & Drink Shop
4,AUXILIADORA,Gym / Fitness Center,Café,Bar
5,AZENHA,Gym / Fitness Center,Motel,Café
6,BELA VISTA,Gym / Fitness Center,Grocery Store,Plaza
7,BELÉM NOVO,Restaurant,Dive Bar,Plaza
8,BOA VISTA,Café,Bistro,Furniture / Home Store
9,BOM FIM,Café,Restaurant,Vegetarian / Vegan Restaurant


In [ ]:
# neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['1st Most Common Venue'] == 'Café') & (neighborhoods_venues_sorted['2st Most Common Venue'] == 'Café']

nvs = neighborhoods_venues_sorted

cafe = nvs[(nvs['1st Most Common Venue']=="Café") | (nvs['2nd Most Common Venue'] =="Café")]
cafe.sort_values(by='1st Most Common Venue')

,Bairro,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
15,CENTRO HISTÓRICO,Buffet,Café,Brazilian Restaurant
8,BOA VISTA,Café,Bistro,Furniture / Home Store
9,BOM FIM,Café,Restaurant,Vegetarian / Vegan Restaurant
32,INDEPENDÊNCIA,Café,Restaurant,Brazilian Restaurant
46,MENINO DEUS,Café,Pastry Shop,Plaza
47,MOINHOS DE VENTO,Café,Brazilian Restaurant,Gym / Fitness Center
4,AUXILIADORA,Gym / Fitness Center,Café,Bar


Agora contar quantos cafés tem por bairro e dividir pelas notas deles.

In [ ]:
cafe_poa = poa_venues[poa_venues["Venue Category"] == "Café"]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
109,TRISTEZA,-30.110418,-51.251529,Matra Café,-30.108751,-51.253157,Café
119,TRISTEZA,-30.110418,-51.251529,Café & Prosa,-30.113582,-51.250809,Café
144,TRISTEZA,-30.110418,-51.251529,Engenho do Café,-30.110191,-51.253077,Café
294,MENINO DEUS,-30.054671,-51.224002,Croasonho,-30.053691,-51.222314,Café
299,MENINO DEUS,-30.054671,-51.224002,Liberty Café,-30.052545,-51.222140,Café


In [ ]:
# cafe_poa = cafe_poa.drop(columns=['', 'Type'])
cafee = cafe_poa.rename(columns={"Venue Latitude": "VenLat", "Venue Longitude": "VenLgn"})
cafee.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,VenLat,VenLgn,Venue Category
109,TRISTEZA,-30.110418,-51.251529,Matra Café,-30.108751,-51.253157,Café
119,TRISTEZA,-30.110418,-51.251529,Café & Prosa,-30.113582,-51.250809,Café
144,TRISTEZA,-30.110418,-51.251529,Engenho do Café,-30.110191,-51.253077,Café
294,MENINO DEUS,-30.054671,-51.224002,Croasonho,-30.053691,-51.222314,Café
299,MENINO DEUS,-30.054671,-51.224002,Liberty Café,-30.052545,-51.222140,Café


In [ ]:
from folium.plugins import HeatMap
lat = cafee.VenLat.tolist()
lng = cafee.VenLgn.tolist()
m = folium.Map(
    location=[-30.0534301, -51.2299346],
    tiles='CartoDB positron',
    zoom_start=12
)
HeatMap(list(zip(lat, lng))).add_to(m)
m

# cartodbdark_matter

### Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 50

poa_grouped_clustering = poa_grouped.drop('Bairro', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(poa_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
poa_merged = df
poa_merged.head()

In [ ]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
poa_merged = poa_merged.join(neighborhoods_venues_sorted.set_index('Bairro'), on='NOME')
 # check the last columns!

In [ ]:
poa_merged

In [ ]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(poa_merged['lat'], poa_merged['lon'], poa_merged['NOME'], poa_merged['New Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=[cluster-1],
        fill=True,
        fill_color=[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [ ]:
poa_merged.loc[poa_merged['New Cluster'] == 0, poa_merged.columns[[1] + list(range(5, poa_merged.shape[1]))]]